In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import re
import PIL
from PIL import Image
import pickle 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten
import random

In [2]:
def jpeg_to_8_bit_greyscale(path, maxsize):
        img = Image.open(path).convert('L')   
        WIDTH, HEIGHT = img.size
        if WIDTH != HEIGHT:
                m_min_d = min(WIDTH, HEIGHT)
                img = img.crop((0, 0, m_min_d, m_min_d))
        img.thumbnail(maxsize, PIL.Image.ANTIALIAS)
        return np.asarray(img)

In [3]:
def load_image_dataset(path_dir, maxsize):
        images = []
        labels = []
        os.chdir(path_dir)
        for file in glob.glob("*.png"):
                img = jpeg_to_8_bit_greyscale(file, maxsize)
                if re.match('Leve.*', file):
                        images.append(img)
                        labels.append(0)

                elif re.match('Carregado.*', file):
                        images.append(img)
                        labels.append(1)

                elif re.match('sobrecarregado.*', file):
                        images.append(img)
                        labels.append(2)
        return (np.asarray(images), np.asarray(labels))

In [4]:
maxsize = 250, 250
def display_images(images, labels):
        plt.figure(figsize=(80, 80))
        grid_size = min(55, len(images))
        for i in range(grid_size):
                plt.subplot(11, 5, i+1)
                plt.xticks([])
                plt.yticks([])
                plt.grid(False)
                plt.imshow(images[i], cmap=plt.cm.binary)
                plt.xlabel(class_names[labels[i]])

In [5]:
(train_images, train_labels) = load_image_dataset(r'C:\Users\labin\OneDrive\Área de Trabalho\Rede Neural\treino', maxsize)
(test_images, test_labels) = load_image_dataset(r'C:\Users\labin\OneDrive\Área de Trabalho\Rede Neural\teste', maxsize)

train_images = train_images / 255
test_images = test_images / 255

print(train_labels)

C:\Users\labin\AppData\Local\Temp\ipykernel_9432\1613596399.py:7: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img.thumbnail(maxsize, PIL.Image.ANTIALIAS)


[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2]


In [6]:
indices = list(range(len(train_labels)))

# Embaralhe os índices
np.random.shuffle(indices)

# Use os índices embaralhados para embaralhar as duas listas
train_images = train_images[indices]
train_labels = train_labels[indices]

class_names = [ 'leve', 'carregado', 'sobrecarregado']

In [7]:
model = Sequential() # 28x28
model.add(Conv2D(32, (3,3), activation='relu',input_shape=(250,250,1) )) #20x24x24
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) #40x8x8
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Conv2D(128, kernel_size=(3,3), activation='relu')) #40x8x8
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(192, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=150, batch_size=43)

Epoch 1/150
4/4 [==============================] - 6s 1s/step - loss: 0.7931 - accuracy: 0.6429
Epoch 2/150
4/4 [==============================] - 6s 1s/step - loss: 0.4878 - accuracy: 0.9143
Epoch 3/150
4/4 [==============================] - 6s 1s/step - loss: 0.4078 - accuracy: 0.9143
Epoch 4/150
4/4 [==============================] - 6s 1s/step - loss: 0.2995 - accuracy: 0.9286
Epoch 5/150
4/4 [==============================] - 6s 1s/step - loss: 0.2723 - accuracy: 0.9286
Epoch 6/150
4/4 [==============================] - 6s 1s/step - loss: 0.1950 - accuracy: 0.9429
Epoch 7/150
4/4 [==============================] - 6s 1s/step - loss: 0.1636 - accuracy: 0.9571
Epoch 8/150
4/4 [==============================] - 6s 1s/step - loss: 0.0996 - accuracy: 0.9857
Epoch 9/150
4/4 [==============================] - 6s 1s/step - loss: 0.0842 - accuracy: 0.9571
Epoch 10/150
4/4 [==============================] - 6s 1s/step - loss: 0.0678 - accuracy: 0.9929
Epoch 11/150
4/4 [=====================

In [11]:
import keras_tuner as kt
def model_builder(hp):
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model = Sequential() # 28x28
  model.add(Conv2D(32, (3,3), activation='relu',input_shape=(250,250,1) )) #20x24x24
  model.add(MaxPooling2D(pool_size=(2,2))) 

  model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) #40x8x8
  model.add(MaxPooling2D(pool_size=(2,2))) 

  model.add(Conv2D(128, kernel_size=(3,3), activation='relu')) #40x8x8
  model.add(MaxPooling2D(pool_size=(2,2))) 

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(tf.keras.layers.Dense(3, activation='softmax'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_images, train_labels, epochs=100, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

FailedPreconditionError: my_dir\intro_to_kt is not a directory

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
acerto = str(test_acc*100)
print(acerto)


1/1 [==============================] - 0s 157ms/step - loss: 11.3714 - accuracy: 0.4286
42.85714328289032


In [9]:
a = model.predict(test_images)
result = []
for x in a:
    x = x.tolist()
    result.append(x.index(max(x)))
print(result)
print(test_labels.tolist())

1/1 [==============================] - 0s 117ms/step
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 2, 2, 2]


In [24]:
from PIL import Image
import os

# Pasta de entrada contendo as imagens originais
pasta_de_entrada = r'C:\Users\labin\OneDrive\Documentos\GitHub\Constru-o-da-Rede-Neural\teste'

# Pasta de saída onde as imagens redimensionadas serão salvas
pasta_de_saida = r'C:\Users\labin\OneDrive\Documentos\GitHub\Constru-o-da-Rede-Neural\aa'

# Tamanho desejado das imagens (300x300)
novo_tamanho = (300, 300)

# Certifique-se de que a pasta de saída exista ou crie-a
if not os.path.exists(pasta_de_saida):
    os.makedirs(pasta_de_saida)

# Lista todos os arquivos na pasta de entrada
arquivos = os.listdir(pasta_de_entrada)

for arquivo in arquivos:
    try:
        # Abre a imagem original
        imagem = Image.open(os.path.join(pasta_de_entrada, arquivo))
        
        # Redimensiona a imagem para o tamanho desejado
        imagem_redimensionada = imagem.resize(novo_tamanho)
        
        # Salva a imagem redimensionada na pasta de saída
        imagem_redimensionada.save(os.path.join(pasta_de_saida, arquivo))
        
        print(f'{arquivo} redimensionado com sucesso!')
    except Exception as e:
        print(f'Erro ao redimensionar {arquivo}: {str(e)}')


Carregado (1).jpeg redimensionado com sucesso!
Carregado (1).jpg redimensionado com sucesso!
Carregado (1).png redimensionado com sucesso!
Leve (1).jpeg redimensionado com sucesso!
Leve (1).jpg redimensionado com sucesso!
Leve (1).png redimensionado com sucesso!
Leve (2).jpeg redimensionado com sucesso!
Leve (2).jpg redimensionado com sucesso!
Leve (2).png redimensionado com sucesso!
Leve (3).jpeg redimensionado com sucesso!
sobrecarregado005.png redimensionado com sucesso!
sobrecarregado006.png redimensionado com sucesso!
sobrecarregado007.png redimensionado com sucesso!


In [25]:
from PIL import Image
import os

pasta_de_entrada = r'C:\Users\labin\OneDrive\Documentos\GitHub\Constru-o-da-Rede-Neural\teste'

# Pasta de saída onde as imagens redimensionadas serão salvas
pasta_de_saida = r'C:\Users\labin\OneDrive\Documentos\GitHub\Constru-o-da-Rede-Neural\aa'

# Certifique-se de que a pasta de saída exista ou crie-a
if not os.path.exists(pasta_de_saida):
    os.makedirs(pasta_de_saida)

# Lista todos os arquivos na pasta de entrada
arquivos = os.listdir(pasta_de_entrada)

for arquivo in arquivos:
    try:
        # Abre a imagem original
        imagem = Image.open(os.path.join(pasta_de_entrada, arquivo))
        
        # Define o formato de saída como PNG
        imagem.save(os.path.join(pasta_de_saida, f'{os.path.splitext(arquivo)[0]}.png'), 'PNG')
        
        print(f'{arquivo} convertido para PNG com sucesso!')
    except Exception as e:
        print(f'Erro ao converter {arquivo} para PNG: {str(e)}')


Carregado (1).jpeg convertido para PNG com sucesso!
Carregado (1).jpg convertido para PNG com sucesso!
Carregado (1).png convertido para PNG com sucesso!
Leve (1).jpeg convertido para PNG com sucesso!
Leve (1).jpg convertido para PNG com sucesso!
Leve (1).png convertido para PNG com sucesso!
Leve (2).jpeg convertido para PNG com sucesso!
Leve (2).jpg convertido para PNG com sucesso!
Leve (2).png convertido para PNG com sucesso!
Leve (3).jpeg convertido para PNG com sucesso!
sobrecarregado005.png convertido para PNG com sucesso!
sobrecarregado006.png convertido para PNG com sucesso!
sobrecarregado007.png convertido para PNG com sucesso!
